In [ ]:
import ray
ray.init(ignore_reinit_error=True, object_store_memory=2 *1024 * 1024 * 1024)#, memory=1 * 1024 * 1024, num_cpus=4)

In [ ]:
import numpy as np
from plangym import AtariEnvironment, ParallelEnvironment
from fragile.ray.swarm import RemoteSwarm, DistributedSwarm, ParamServer
from fragile.ray.demos import DistributedRam

from fragile.atari.env import AtariEnv
from fragile.core.dt_sampler import GaussianDt
from fragile.core.env import DiscreteEnv
from fragile.core.models import RandomDiscrete
from fragile.core.states import States
from fragile.core.swarm import Swarm
from fragile.core.tree import HistoryTree
from fragile.core.walkers import Walkers

def create_swarm():
    
    env = AtariEnvironment(name="MsPacman-ram-v0",
            clone_seeds=True,
            autoreset=True)

    dt = GaussianDt(min_dt=3, max_dt=1000, loc_dt=4, scale_dt=2)
    swarm = Swarm(
        model=lambda x: RandomDiscrete(x, dt_sampler=dt),
        walkers=Walkers,
        env=lambda: AtariEnv(env),
        n_walkers=25,
        max_iters=10000000000000000,
        prune_tree=True,
        reward_scale=2,
        minimize=False,
        tree=None, #HistoryTree,
    )
    return swarm
import hvplot.pandas

In [ ]:
rs = DistributedRam(create_swarm, n_swarms=4, max_iters_ray=10000, log_every=10, n_comp_add=3, n_param_servers=1)

In [ ]:
dmap = rs.plot()
dmap

In [ ]:
rs.run_swarm()